In [75]:
import sys, os, re
import pandas as pd
import numpy as np
import json
import requests
import csv


In [76]:
string_name = pd.read_csv('data/9606.protein.info.v11.5.txt', sep="\t")

In [77]:
string_name.head()

,#string_protein_id,preferred_name,protein_size,annotation
0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...
2,9606.ENSP00000001008,FKBP4,459,Peptidyl-prolyl cis-trans isomerase FKBP4; Imm...
3,9606.ENSP00000001146,CYP26B1,512,Cytochrome P450 26B1; Involved in the metaboli...
4,9606.ENSP00000002125,NDUFAF7,441,"Protein arginine methyltransferase NDUFAF7, mi..."


In [78]:
STRING_df_dict = {}

string_name_dict = string_name.to_dict('split')

for i in range(len(string_name_dict['data'])):
    STRING_df_dict[string_name_dict['data'][i][0]] = {"name": string_name_dict['data'][i][1], "description": string_name_dict['data'][i][3].split(";")[0].strip()}

del string_name, string_name_dict

In [79]:
##string

string_link = pd.read_csv('data/9606.protein.links.detailed.v11.5.txt', sep=" ")

string_link.head()

,protein1,protein2,neighborhood,fusion,cooccurence,coexpression,experimental,database,textmining,combined_score
0,9606.ENSP00000000233,9606.ENSP00000379496,0,0,0,54,0,0,144,155
1,9606.ENSP00000000233,9606.ENSP00000314067,0,0,0,0,180,0,61,197
2,9606.ENSP00000000233,9606.ENSP00000263116,0,0,0,62,152,0,101,222
3,9606.ENSP00000000233,9606.ENSP00000361263,0,0,0,0,161,0,64,181
4,9606.ENSP00000000233,9606.ENSP00000409666,0,0,0,82,213,0,72,270


In [80]:
combine_cutoff_score = 900

string_link_dict = string_link[string_link["combined_score"] >= combine_cutoff_score].to_dict('split')

link_set = set()

STRING_to_add = {}

for i in range(len(string_link_dict['data'])):
    protein_1 = string_link_dict['data'][i][0]
    protein_2 = string_link_dict['data'][i][1]

    #print (protein_1, protein_2)

    if "name" in STRING_df_dict[protein_1] and "name" in STRING_df_dict[protein_2]:
        protein_name_1 = STRING_df_dict[protein_1]["name"]
        protein_name_2 = STRING_df_dict[protein_2]["name"]

        t = tuple(sorted([protein_name_1, protein_name_2]))
        link_set.add(t)

        STRING_to_add[protein_name_1.lower()] = protein_name_1
        STRING_to_add[protein_name_2.lower()] = protein_name_2
    else:
        print ("No name for protein: ", protein_1, protein_2)
del string_link_dict, string_link

In [81]:
len(link_set)

123577

In [82]:
len(STRING_to_add)

11813

Now correct the names in hetionet.gene to STRING

In [83]:
json_connections = set()


with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    id_name = {}

    nodes = []
    for g in json_data["nodes"]:
        if g["kind"] == "Gene":
            if g["name"].lower() in STRING_to_add:
                g["name"] = STRING_to_add[g["name"].lower()]

        id_name[g["identifier"]] = g["name"]
        
        if "data" in g:
            del g["data"]
        nodes.append(g)

    with open('data/hetionet_string_corrected_gene_name_nodes.json', 'w') as outfile:
        json.dump(nodes, outfile, indent=2)

    edges = []
    for edge in json_data["edges"]:
        id_1 = edge["source_id"][1]
        name_1 = id_name[id_1]
        edge["source_id"][1] = name_1
        
        id_2 = edge["target_id"][1]
        name_2 = id_name[id_2]
        edge["target_id"][1] = name_2

        if "data" in edge:
            del edge["data"]
    
        edges.append(edge)

    with open('data/hetionet_string_corrected_gene_name_edges.json', 'w') as outfile:
        json.dump(edges, outfile, indent=2)

Now correct the names with kegg

In [84]:
df = pd.read_csv("disease.csv")

string_name_dict = df.to_dict('split')

kegg_disease_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_disease_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]


df = pd.read_csv("drug.csv")

string_name_dict = df.to_dict('split')

kegg_compound_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_compound_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]


df = pd.read_csv("gene_tmp.csv")

string_name_dict = df.to_dict('split')

kegg_gene_name = {}

for i in range(len(string_name_dict['data'])):
    kegg_gene_name[string_name_dict['data'][i][1].lower()] = string_name_dict['data'][i][1]




with open('data/hetionet_string_corrected_gene_name_nodes.json', 'r') as f:
    nodes = json.load(f)

    new_nodes = []
    for node in nodes:
        if node["kind"] == "Disease":
            if node["name"].lower() in kegg_disease_name:
                node["name"] = kegg_disease_name[node["name"].lower()]
        elif node["kind"] == "Compound":
            if node["name"].lower() in kegg_compound_name:
                node["name"] = kegg_compound_name[node["name"].lower()]
        elif node["kind"] == "Gene":
            if node["name"].lower() in kegg_gene_name:
                node["name"] = kegg_gene_name[node["name"].lower()]
        
        new_nodes.append(node)

        
    with open('data/hetionet_kegg_corrected_gene_name_nodes.json', 'w') as outfile:
        json.dump(new_nodes, outfile, indent=2)


with open('data/hetionet_string_corrected_gene_name_edges.json', 'r') as f:
    edges = json.load(f)

    new_edges = []
    for edge in edges:
        if edge["source_id"][0] == "Disease":
            if edge["source_id"][1].lower() in kegg_disease_name:
                edge["source_id"][1] = kegg_disease_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Disease":
            if edge["target_id"][1].lower() in kegg_disease_name:
                edge["target_id"][1] = kegg_disease_name[edge["target_id"][1].lower()]
        

        if edge["source_id"][0] == "Compound":
            if edge["source_id"][1].lower() in kegg_compound_name:
                edge["source_id"][1] = kegg_compound_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Compound":
            if edge["target_id"][1].lower() in kegg_compound_name:
                edge["target_id"][1] = kegg_compound_name[edge["target_id"][1].lower()]

        
        if edge["source_id"][0] == "Gene":
            if edge["source_id"][1].lower() in kegg_gene_name:
                edge["source_id"][1] = kegg_gene_name[edge["source_id"][1].lower()]

        if edge["target_id"][0] == "Gene":
            if edge["target_id"][1].lower() in kegg_gene_name:
                edge["target_id"][1] = kegg_gene_name[edge["target_id"][1].lower()]
        
        new_edges.append(edge)

        
    with open('data/hetionet_kegg_corrected_gene_name_edges.json', 'w') as outfile:
        json.dump(new_edges, outfile, indent=2)

save string links and nodes

In [85]:
STRING_to_add.values()
string_name = pd.read_csv('data/9606.protein.info.v11.5.txt', sep="\t")

print (len(string_name))
#df_kegg_disease_diff_hetionet = df_kegg_disease[~df_kegg_disease["name"].str.lower().isin(hetionet_disease_name)]
#print (string_name.head())

string_name = string_name[string_name["preferred_name"].isin(STRING_to_add.values())]
string_name["description"] = string_name["annotation"].str.split(";").str[0]

string_name.drop(["#string_protein_id", "protein_size", "annotation"], axis = 1, inplace = True)

string_name.rename({'preferred_name': 'name'}, axis='columns', inplace=True)

print (string_name.head())

print (len(string_name))

string_name.to_csv("data/string_to_add_nodes.csv", index = False, quoting=csv.QUOTE_ALL)


19566
      name                                        description
0     ARF5                          ADP-ribosylation factor 5
1     M6PR      Cation-dependent mannose-6-phosphate receptor
2    FKBP4          Peptidyl-prolyl cis-trans isomerase FKBP4
3  CYP26B1                               Cytochrome P450 26B1
4  NDUFAF7  Protein arginine methyltransferase NDUFAF7, mi...
11814


In [86]:
content = []
"""
{
    "source_id": [
      "Gene",
      "CALM1"
    ],
    "target_id": [
      "Gene",
      "VHL"
    ],
    "kind": "interacts",
    "direction": "both"
  },
"""


for new_edge in link_set:
    content.append({"source_id": ["Gene", new_edge[0]], "target_id": ["Gene", new_edge[1]], "kind": "STRING"})

with open('data/string_to_add_edges.json', 'w') as outfile:
    json.dump(content, outfile, indent=2)

In [87]:
with open('data/hetionet_kegg_corrected_gene_name_edges.json', 'r') as f:
    edges = json.load(f)

    print (len(edges))

2250197


Kegg Disease

In [88]:

#<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">ICD10CM:N46.0</oboInOwl:hasDbXref>
#<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">MESH:D006509</oboInOwl:hasDbXref>

rx_icd10 = re.compile(r'<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">ICD10CM:(\S+)</oboInOwl:hasDbXref>')
rx_mesh = re.compile(r'<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">MESH:(\S+)</oboInOwl:hasDbXref>')
rx_kegg = re.compile(r'<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">KEGG:(\S+)</oboInOwl:hasDbXref>')
rx_omid = re.compile(r'<oboInOwl:hasDbXref rdf:datatype="http://www.w3.org/2001/XMLSchema#string">OMIM:(\S+)</oboInOwl:hasDbXref>')

disease_id = {}

with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    for node in json_data["nodes"]:
        if node["kind"] == "Disease" and "data" in node and "url" in node["data"]:
            url = node["data"]["url"]
            r = requests.get(url)
            #print (url)

            #print (r.text)

            s_icd10 = rx_icd10.findall(r.text)

            for s in s_icd10:
                if node["identifier"] not in disease_id:
                    disease_id[node["identifier"]] = {}
                if "icd10" not in disease_id[node["identifier"]]:
                    disease_id[node["identifier"]]["icd10"] = []
                disease_id[node["identifier"]]["icd10"].append(s)


            s_mesh = rx_mesh.findall(r.text)

            for s in s_mesh:
                if node["identifier"] not in disease_id:
                    disease_id[node["identifier"]] = {}
                
                if "mesh" not in disease_id[node["identifier"]]:
                    disease_id[node["identifier"]]["mesh"] = []

                disease_id[node["identifier"]]["mesh"].append(s)
                

            s_kegg = rx_kegg.findall(r.text)

            for s in s_kegg:
                if node["identifier"] not in disease_id:
                    disease_id[node["identifier"]] = {}
                
                if "omid" not in disease_id[node["identifier"]]:
                    disease_id[node["identifier"]]["kegg"] = []
                disease_id[node["identifier"]]["kegg"].append(s)
                

            s_omid = rx_omid.findall(r.text)

            for s in s_omid:
                if node["identifier"] not in disease_id:
                    disease_id[node["identifier"]] = {}
                
                if "omid" not in disease_id[node["identifier"]]:
                    disease_id[node["identifier"]]["omid"] = []
                disease_id[node["identifier"]]["omid"].append(s)
                

In [89]:
disease_id

{'DOID:14227': {'icd10': ['N46.0'], 'mesh': ['D053713']},
 'DOID:9352': {'icd10': ['E11'],
  'mesh': ['D003924'],
  'kegg': ['04930'],
  'omid': ['125853', '601283', '601407', '603694', '608036']},
 'DOID:8778': {'icd10': ['K50.1'], 'mesh': ['D003424']},
 'DOID:5612': {'icd10': ['C72.0'], 'mesh': ['D013120', 'D013125']},
 'DOID:363': {'icd10': ['C55'], 'mesh': ['D014594']},
 'DOID:11054': {'icd10': ['C67'],
  'mesh': ['D001749'],
  'kegg': ['05219'],
  'omid': ['109800']},
 'DOID:1793': {'icd10': ['C25.0', 'C25.1', 'C25.2'],
  'mesh': ['D010190'],
  'kegg': ['05212']},
 'DOID:8850': {'icd10': ['C08']},
 'DOID:0060119': {'mesh': ['D010610']},
 'DOID:10763': {'icd10': ['I10'], 'mesh': ['D006973']},
 'DOID:3312': {'icd10': ['F31'], 'mesh': ['D001714']},
 'DOID:418': {'icd10': ['M34.0'], 'mesh': ['D012595'], 'omid': ['181750']},
 'DOID:2377': {'icd10': ['G35'],
  'mesh': ['D009103'],
  'omid': ['612594', '612595', '612596']},
 'DOID:4606': {'icd10': ['C24.0'], 'mesh': ['D001650']},
 'DOID:

In [90]:
with open("data/hetionet_disease_dblink.json", "w") as f:
    json.dump(disease_id, f, indent=2)

In [94]:
with open('data/to_s3/hetionet_kegg_corrected_gene_name_nodes.json', 'r') as f:
    nodes = json.load(f)

    new_nodes = []
    for node in nodes:
        if node["kind"] == "Disease":
            if node["identifier"] in disease_id and "icd10" in disease_id[node["identifier"]]:
                node["icd10"] = disease_id[node["identifier"]]["icd10"]
                print (node)
        
        new_nodes.append(node)

        
    with open('data/hetionet_kegg_corrected_gene_name_nodes_2.json', 'w') as outfile:
        json.dump(new_nodes, outfile, indent=2)

{'kind': 'Disease', 'identifier': 'DOID:14227', 'name': 'Azoospermia', 'icd10': ['N46.0']}
{'kind': 'Disease', 'identifier': 'DOID:9352', 'name': 'Type 2 diabetes mellitus', 'icd10': ['E11']}
{'kind': 'Disease', 'identifier': 'DOID:8778', 'name': "Crohn's disease", 'icd10': ['K50.1']}
{'kind': 'Disease', 'identifier': 'DOID:5612', 'name': 'spinal cancer', 'icd10': ['C72.0']}
{'kind': 'Disease', 'identifier': 'DOID:363', 'name': 'uterine cancer', 'icd10': ['C55']}
{'kind': 'Disease', 'identifier': 'DOID:11054', 'name': 'urinary bladder cancer', 'icd10': ['C67']}
{'kind': 'Disease', 'identifier': 'DOID:1793', 'name': 'Pancreatic cancer', 'icd10': ['C25.0', 'C25.1', 'C25.2']}
{'kind': 'Disease', 'identifier': 'DOID:8850', 'name': 'Salivary gland cancer', 'icd10': ['C08']}
{'kind': 'Disease', 'identifier': 'DOID:10763', 'name': 'hypertension', 'icd10': ['I10']}
{'kind': 'Disease', 'identifier': 'DOID:3312', 'name': 'Bipolar disorder', 'icd10': ['F31']}
{'kind': 'Disease', 'identifier': 'DO

In [ ]:
url = "http://www.ontobee.org/ontology/DOID?iri=http://purl.obolibrary.org/obo/DOID_7693"
r = requests.get(url)
print (url)

#print (r.text)

s_omid = rx_omid.findall(r.text)

for s in s_omid:
    print (s)

http://www.ontobee.org/ontology/DOID?iri=http://purl.obolibrary.org/obo/DOID_7693
100070
609782
611891
614375


In [ ]:
hetionet_kegg = {}

with open('data/disease_kegg_id.json', 'r') as f:
    kegg_disease = json.load(f)

    with open('data/disease_kegg.json', 'r') as f1:
        hetionet_disease = json.load(f1)

        for disease in hetionet_disease:
            
            icd10 = []
            mesh = []
            kegg_id = []
            omid = []
            if "icd10" in hetionet_disease[disease]:
                icd10 = hetionet_disease[disease]["icd10"]
            
            if "mesh" in hetionet_disease[disease]:
                mesh = hetionet_disease[disease]["mesh"]

            if "kegg" in hetionet_disease[disease]:
                kegg_id = hetionet_disease[disease]["kegg"]
            
            if "omid" in hetionet_disease[disease]:
                omid = hetionet_disease[disease]["omid"]

            for kegg in kegg_disease:
                
                if "ICD-10" in kegg_disease[kegg]:
                    #print (icd10, kegg_disease[kegg]["ICD-10"])
                    for i in icd10:
                        if i in kegg_disease[kegg]["ICD-10"]:
                            hetionet_kegg[disease] = kegg
                
                if "MeSH" in kegg_disease[kegg]:
                    for m in mesh:
                        if m in kegg_disease[kegg]["MeSH"]:
                        
                            hetionet_kegg[disease] = kegg
                
                if "kegg" in kegg_disease[kegg]:
                    for k in kegg_id:
                        if k in kegg_disease[kegg]["kegg"]:
                            hetionet_kegg[disease] = kegg
                
                if "OMIM" in kegg_disease[kegg]:
                    for o in omid:
                        if o in kegg_disease[kegg]["OMIM"]:
                            hetionet_kegg[disease] = kegg

In [ ]:
kegg_icd10 = {}
with open('data/disease_kegg_id.json', 'r') as f:
    kegg_disease = json.load(f)
    for kegg in kegg_disease:
        if "ICD-10" in kegg_disease[kegg]:
            kegg_icd10[kegg] = kegg_disease[kegg]["ICD-10"]

In [ ]:
hetionet_icd10 = {}
with open('data/disease_kegg.json', 'r') as f1:
    hetionet_disease = json.load(f1)

    for disease in hetionet_disease:
        if "icd10" in hetionet_disease[disease]:
            hetionet_icd10[disease] = hetionet_disease[disease]["icd10"]
        
        else:
            if disease in hetionet_kegg:
                kegg = hetionet_kegg[disease]

                if kegg in kegg_icd10:
                    icd = kegg_icd10[kegg]

                    hetionet_icd10[disease] = icd

In [ ]:
len(kegg_icd10)


2297

In [ ]:
len(hetionet_icd10)

118

In [ ]:
with open("data/hetionet_icd10.json", "w") as f:
    json.dump(hetionet_icd10, f, indent=2)

In [ ]:
with open("data/kegg_icd10.json", "w") as f:
    json.dump(kegg_icd10, f, indent=2)

In [109]:
df_kegg_disease = pd.read_csv("disease.csv")

In [110]:
with open("data/kegg_icd10.json") as f:
    kegg_icd10 = json.load(f)

    df_kegg_disease["icd10"] = df_kegg_disease["ko"].apply(lambda x: str(kegg_icd10[x]) if x in kegg_icd10 else np.nan)

In [113]:
df_kegg_disease.to_csv("disease.csv", index=False, quoting=csv.QUOTE_ALL)

In [115]:
kegg_disease_id_name_dict = df_kegg_disease.to_dict('split')

name_dict = {}

for i in range(len(kegg_disease_id_name_dict['data'])):
    name_dict[kegg_disease_id_name_dict['data'][i][0]] = kegg_disease_id_name_dict['data'][i][1]



hetionet_disease_name = {}
with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    id_name = {}
    for g in json_data["nodes"]:
        if g["kind"] == "Disease":
            if g["identifier"] in hetionet_kegg and hetionet_kegg[g["identifier"]] in name_dict:
                hetionet_disease_name[g["identifier"]] = name_dict[hetionet_kegg[g["identifier"]]].lower()
            else:
                hetionet_disease_name[g["identifier"]] = g["name"].lower()

In [116]:
with open("data/hetionet_disease_id_kegg_name.json", 'w') as f:
    json.dump(hetionet_disease_name, f, indent=2)

In [61]:
hetionet_kegg = {}

kegg_hetionet = {}

with open('data/disease_kegg_id.json', 'r') as f:
    kegg_disease = json.load(f)

    with open('data/disease_kegg.json', 'r') as f1:
        hetionet_disease = json.load(f1)

        for disease in hetionet_disease:
            
            icd10 = []
            mesh = []
            kegg_id = []
            omid = []
            if "icd10" in hetionet_disease[disease]:
                icd10 = hetionet_disease[disease]["icd10"]
            
            if "mesh" in hetionet_disease[disease]:
                mesh = hetionet_disease[disease]["mesh"]

            if "kegg" in hetionet_disease[disease]:
                kegg_id = hetionet_disease[disease]["kegg"]
            
            if "omid" in hetionet_disease[disease]:
                omid = hetionet_disease[disease]["omid"]

            for kegg in kegg_disease:
                
                if "ICD-10" in kegg_disease[kegg]:
                    #print (icd10, kegg_disease[kegg]["ICD-10"])
                    for i in icd10:
                        if i in kegg_disease[kegg]["ICD-10"]:
                            hetionet_kegg[disease] = kegg
                            kegg_hetionet[kegg] = hetionet_disease_name[disease]
                
                if "MeSH" in kegg_disease[kegg]:
                    for m in mesh:
                        if m in kegg_disease[kegg]["MeSH"]:
                        
                            hetionet_kegg[disease] = kegg
                            kegg_hetionet[kegg] = hetionet_disease_name[disease]
                
                if "kegg" in kegg_disease[kegg]:
                    for k in kegg_id:
                        if k in kegg_disease[kegg]["kegg"]:
                            hetionet_kegg[disease] = kegg
                            kegg_hetionet[kegg] = hetionet_disease_name[disease]
                
                if "OMIM" in kegg_disease[kegg]:
                    for o in omid:
                        if o in kegg_disease[kegg]["OMIM"]:
                            hetionet_kegg[disease] = kegg
                            kegg_hetionet[kegg] = hetionet_disease_name[disease]

In [23]:
hetionet_disease_name = set()
with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    id_name = {}
    for g in json_data["nodes"]:
        if g["kind"] == "Disease":
            hetionet_disease_name.add(g["name"].lower())

df_kegg_disease = pd.read_csv("disease.csv")

print (len(df_kegg_disease))

df_kegg_disease_diff_hetionet = df_kegg_disease[~df_kegg_disease["name"].str.lower().isin(hetionet_disease_name)]

print (len(df_kegg_disease_diff_hetionet))

2545
2496


In [25]:
hetionet_compound_name = set()
with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    id_name = {}
    for g in json_data["nodes"]:
        if g["kind"] == "Compound":
            hetionet_compound_name.add(g["name"].lower())

df_kegg_drug = pd.read_csv("drug.csv")

print (len(df_kegg_drug))

df_kegg_drug_diff_hetionet = df_kegg_drug[~df_kegg_drug["name"].str.lower().isin(hetionet_compound_name)]

print (len(df_kegg_drug_diff_hetionet))

1389
1031


In [26]:
df_kegg_drug_diff_hetionet.to_csv("data/df_kegg_drug_diff_hetionet.csv", index=False, quoting=csv.QUOTE_ALL)

In [65]:
drugs = pd.read_csv("drug.csv")

In [69]:
rx_drugbank = re.compile(r"DrugBank:\s+(\w+)")

for i in range(len(drugs)):
    ko = drugs.loc[i, "ko"]
    
    url = "http://rest.kegg.jp/get/" + ko
    r = requests.get(url)

    drugbanks = rx_drugbank.findall(r.text)

    if len(drugbanks) > 0:
        #print (ko, d)

        drugs.loc[i, "drugbank"] = drugbanks[0]


In [70]:
drugs.head()

,ko,name,drugbank
0,D00180,Alprostadil,DB00770
1,D06199,Trabectedin,NaN
2,D01275,Doxorubicin hydrochloride,NaN
3,D05380,Pazopanib hydrochloride,NaN
4,D09939,Olaratumab,NaN


In [71]:
drugs["drugbank"].isnull().sum()

738

In [72]:
len(drugs)

1389

In [73]:
drugs.to_csv("data/drugs_with_drugbank.tsv", sep="\t", index=False)

In [76]:
id_name = {}

new_hetionet = {"nodes": [], "edges": []}

with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    for node in json_data["nodes"]:
        
        if "identifier" in node:
            id_name[node["identifier"]] = node["name"]
        
        new_hetionet["nodes"].append(node)

    for edge in json_data["edges"]:
        if "source_id" in edge and "target_id" in edge and edge["source_id"][1] in id_name and edge["target_id"][1] in id_name:
            edge["source_id"][1] = id_name[edge["source_id"][1]]
            edge["target_id"][1] = id_name[edge["target_id"][1]]

            new_hetionet["edges"].append(edge)

In [77]:
with open("data/new_hetionet.json", "w") as f:
    json.dump(new_hetionet, f, indent=2)

In [ ]:
with open("data/string_name_dict.json") as f:
    string_name_dict = json.load(f)

    

In [129]:
with open('data/hetionet-v1.0.json', 'r') as f:
    json_data = json.load(f)

    new_nodes = []
    for g in json_data["nodes"]:
        if g["kind"] == "Disease":
            if g["identifier"] in hetionet_kegg and hetionet_kegg[g["identifier"]] in name_dict:
                g["name"] = name_dict[hetionet_kegg[g["identifier"]]].lower()
            else:
                g["name"] = g["name"].lower()
            if "data" in g:
                del g["data"]
            new_nodes.append(g)
        else:
            if "data" in g:
                del g["data"]
            new_nodes.append(g)
    
    with open('data/hetionet-new_nodes.json', 'w') as f:
        json.dump(new_nodes, f, indent=2)


In [3]:
with open('data/hetionet-new_nodes.json', 'r') as f:
    nodes = json.load(f)

    id_name = {}

    for node in nodes:
        id_name[node["identifier"]] = node["name"]
    
    new_edges = []

    with open('data/hetionet-v1.0.json', 'r') as f:
        json_data = json.load(f)

        for edge in json_data["edges"]:
            if "source_id" in edge and "target_id" in edge and edge["source_id"][1] in id_name and edge["target_id"][1] in id_name:
                edge["source_id"][1] = id_name[edge["source_id"][1]]
                edge["target_id"][1] = id_name[edge["target_id"][1]]

                if "data" in edge:
                    del edge["data"]

                new_edges.append(edge)

    with open('data/hetionet-new_edges.json', 'w') as o:
        json.dump(new_edges, o, indent=2)

In [4]:
len(new_edges)

2250197